| Size     | Layers | Width | Heads | Parameters | English-only                                         | Multilingual                                        |
|----------|--------|-------|-------|------------|------------------------------------------------------|-----------------------------------------------------|
| tiny     | 4      | 384   | 6     | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)    |
| base     | 6      | 512   | 8     | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)     |
| small    | 12     | 768   | 12    | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)    |
| medium   | 24     | 1024  | 16    | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium)   |
| large    | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)    |
| large-v2 | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v2) |
| large-v3 | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large-v3) |


We can verify that we've been assigned a GPU and view its specifications:

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  2 17:50:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.47                 Driver Version: 572.47         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              3W /  105W |    1140MiB /   6141MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

ERROR: To modify pip, please run the following command:
C:\Users\sirip\Anaconda3\envs\elasticsearch\python.exe -m pip install --upgrade --quiet pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


In [4]:
!pip install ipywidgets

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.3 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 4.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "th", split="train",trust_remote_code=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "th", split="test",trust_remote_code=True)

print(common_voice)

n_shards.json:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

th_train_0.tar:   0%|          | 0.00/821M [00:00<?, ?B/s]

th_dev_0.tar:   0%|          | 0.00/319M [00:00<?, ?B/s]

th_test_0.tar:   0%|          | 0.00/333M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [16]:
from datasets import load_dataset

common_voice = load_dataset("CMKL/Porjai-Thai-voice-dataset-central", split="train[0:500]", trust_remote_code=True)

print(common_voice)


Dataset({
    features: ['audio', 'sentence', 'utterance'],
    num_rows: 500
})


In [22]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("CMKL/Porjai-Thai-voice-dataset-central", split="train[0:400]", trust_remote_code=True)
common_voice["test"] = load_dataset("CMKL/Porjai-Thai-voice-dataset-central", split="train[400:500]", trust_remote_code=True)

In [23]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'utterance'],
        num_rows: 400
    })
    test: Dataset({
        features: ['audio', 'sentence', 'utterance'],
        num_rows: 100
    })
})


In [ ]:
common_voice

In [1]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

NameError: name 'common_voice' is not defined

In [24]:
common_voice = common_voice.remove_columns(["utterance"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 400
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 100
    })
})


In [20]:
common_voice["train"] = common_voice["train"].select(range(int(len(common_voice["train"]) * 0.2)))
common_voice["test"] = common_voice["test"].select(range(int(len(common_voice["test"]) * 0.2)))

In [4]:
print(common_voice)

Dataset({
    features: ['audio', 'sentence', 'utterance'],
    num_rows: 200
})


## Prepare Feature Extractor, Tokenizer and Data

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("nectec/Pathumma-whisper-th-large-v3")

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

c:\Users\sirip\Anaconda3\envs\elasticsearch\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sirip\.cache\huggingface\hub\models--openai--whisper-large-v3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [38]:
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 128,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [34]:
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 128,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [39]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("nectec/Pathumma-whisper-th-large-v3", language="Thai", task="transcribe")

In [40]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("nectec/Pathumma-whisper-th-large-v3", language="Thai", task="transcribe")

### Prepare Data

In [28]:
print(common_voice["train"][0])

{'audio': {'path': 'thai-central_000000.mp3', 'array': array([7.84463055e-08, 7.76523947e-08, 7.50967999e-08, ...,
       9.49971552e-04, 9.58466379e-04, 8.03229341e-04]), 'sampling_rate': 16000}, 'sentence': 'ทีม จาก อิสราเอล ไม่ ควร ได้ เป็น เจ้าบ้าน ใน เกม ยูฟ่า คัพ'}


In [29]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [30]:
print(common_voice["train"][0])

{'audio': {'path': 'thai-central_000000.mp3', 'array': array([7.84463055e-08, 7.76523947e-08, 7.50967999e-08, ...,
       9.49971552e-04, 9.58466379e-04, 8.03229341e-04]), 'sampling_rate': 16000}, 'sentence': 'ทีม จาก อิสราเอล ไม่ ควร ได้ เป็น เจ้าบ้าน ใน เกม ยูฟ่า คัพ'}


In [57]:
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 128,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [58]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [60]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"])

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [62]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("nectec/Pathumma-whisper-th-large-v3")

In [63]:
model.generation_config.language = "Thai"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [64]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [65]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [66]:
import evaluate

metric = evaluate.load("wer")

In [67]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [72]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-th-large-v3",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=20,
    max_steps=300,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=200,
    save_steps=20,
    eval_steps=20,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    remove_unused_columns=False
)

c:\Users\sirip\Anaconda3\envs\elasticsearch\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [73]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

C:\Users\sirip\AppData\Local\Temp\ipykernel_27408\240849381.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [74]:
processor.save_pretrained(training_args.output_dir)

[]

In [76]:
import torch

torch.cuda.empty_cache()

In [75]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 11.56 GiB is allocated by PyTorch, and 721.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
kwargs = {
    "dataset": "CMKL/Porjai-Thai-voice-dataset-central",  # a 'pretty' name for the training dataset
    "language": "th",
    "model_name": "whisper-th-large-v3",  # a 'pretty' name for our model
    "finetuned_from": "nectec/Pathumma-whisper-th-large-v3",
    "tasks": "automatic-speech-recognition",
}

In [78]:
trainer.push_to_hub(**kwargs)

c:\Users\sirip\Anaconda3\envs\elasticsearch\Lib\site-packages\transformers\modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

events.out.tfevents.1740918139.BOI.27408.0:   0%|          | 0.00/11.7k [00:00<?, ?B/s]

events.out.tfevents.1740918294.BOI.27408.1:   0%|          | 0.00/5.87k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

events.out.tfevents.1740918530.BOI.27408.2:   0%|          | 0.00/5.87k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Siripat/whisper-th-large-v3/commit/f1db65f0a71a0c39fa99c0460112bba6dd0544a6', commit_message='End of training', commit_description='', oid='f1db65f0a71a0c39fa99c0460112bba6dd0544a6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Siripat/whisper-th-large-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='Siripat/whisper-th-large-v3'), pr_revision=None, pr_num=None)

## Building a Demo

In [49]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="Siripat/whisper-small-th")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small thai",
    description="Realtime demo for thai speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

Device set to use cuda:0


TypeError: Audio.__init__() got an unexpected keyword argument 'source'